# Determination of Mutual Commutativity for LICOP terms

This is the notebook referred to in Appendix C of Wang, et al. *Planted solutions in quantum chemistry: a framework for generating large, non-trivial
Hamiltonians with known ground state*, arXiv:XXXX.XXXXX

In [1]:
from openfermion.ops import MajoranaOperator
import itertools
import numpy as np

In [2]:
# Define the Majorana operators for 8 different spin-orbitals
# g_0 corresponds to P0 and g_1 to P1, etc. as defined in Eqn. C1 of the paper.
# Defining the Majorana operators explictly does not cause loss of generality, 
# as there are only up to 8 spin-orbitals involved when considering commutativity, 
# regardless of the number of spin-orbitals in the system as a whole.

g_0 = MajoranaOperator(term=(0,), coefficient=1)
g_1 = MajoranaOperator(term=(1,), coefficient=1)
g_2 = MajoranaOperator(term=(2,), coefficient=1)
g_3 = MajoranaOperator(term=(3,), coefficient=1)

g_4 = MajoranaOperator(term=(4,), coefficient=1)
g_5 = MajoranaOperator(term=(5,), coefficient=1)
g_6 = MajoranaOperator(term=(6,), coefficient=1)
g_7 = MajoranaOperator(term=(7,), coefficient=1)

g_8 = MajoranaOperator(term=(8,), coefficient=1)
g_9 = MajoranaOperator(term=(9,), coefficient=1)
g_10 = MajoranaOperator(term=(10,), coefficient=1)
g_11 = MajoranaOperator(term=(11,), coefficient=1)

g_12 = MajoranaOperator(term=(12,), coefficient=1)
g_13 = MajoranaOperator(term=(13,), coefficient=1)
g_14 = MajoranaOperator(term=(14,), coefficient=1)
g_15 = MajoranaOperator(term=(15,), coefficient=1)

g_list = [g_0, g_1, g_2, g_3, g_4, g_5, g_6, g_7,
          g_8, g_9, g_10, g_11, g_12, g_13, g_14, g_15]

In [3]:
orbital_majorana_dict = {
    "p": [0, 1],
    "q": [2, 3],
    "r": [4, 5],
    "s": [6, 7],
    "t": [8, 9],
    "u": [10, 11],
    "v": [12, 13],
    "w": [14, 15],
}



def make_excitation_operator(p, q):
    # a_dag_p = (g_0 - 1.j*g_1)/2
    # a_q = (g_2 + 1.j*g_3)/2
    majorana_indices_dagger = orbital_majorana_dict[p]
    majorana_indices = orbital_majorana_dict[q]
    a_dag_p = (
        g_list[majorana_indices_dagger[0]] - 1.0j * g_list[majorana_indices_dagger[1]]
    ) / 2
    a_q = (g_list[majorana_indices[0]] + 1.0j * g_list[majorana_indices[1]]) / 2
    return a_dag_p * a_q

def eight_fold_orbital_order(orbital_indices):
    # return a list of the 8 standard arrangements of the 4 orbitals
    return [
        (orbital_indices[0], orbital_indices[1], orbital_indices[2], orbital_indices[3]), # pq rs; 0123
        (orbital_indices[0], orbital_indices[1], orbital_indices[3], orbital_indices[2]), # pq sr; 0132
        (orbital_indices[1], orbital_indices[0], orbital_indices[2], orbital_indices[3]), # qp rs; 1023
        (orbital_indices[1], orbital_indices[0], orbital_indices[3], orbital_indices[2]), # qp sr; 1032
        (orbital_indices[2], orbital_indices[3], orbital_indices[0], orbital_indices[1]), # rs pq; 2301
        (orbital_indices[2], orbital_indices[3], orbital_indices[1], orbital_indices[0]), # rs qp; 2310
        (orbital_indices[3], orbital_indices[2], orbital_indices[0], orbital_indices[1]), # sr pq; 3201
        (orbital_indices[3], orbital_indices[2], orbital_indices[1], orbital_indices[0]), # sr qp; 3210
    ]


# Mutual commutativity within G^PR_QS

In [4]:
# Verify that all commutators are zero within a G^PR_QS term by iterating overall all possible selections of identical spin orbital indices
choose_list = [1, 2, 3, 4]
orbital_indices = ["p", "q", "r", "s"]

global_all_commutators_zero = True
global_num_non_zero_commutators = 0
for k_val in choose_list:
    combination_iterator = itertools.combinations(orbital_indices, k_val)
    
    # If orbital_indices in combination, make them the same
    for iter, combination in enumerate(combination_iterator):
        print("Combination:", combination)
        new_orbital_order = np.copy(orbital_indices)
        new_orbital_val = combination[0]
        for orbital in combination:
            new_orbital_order[new_orbital_order == orbital] = new_orbital_val
        print("New orbital order:", new_orbital_order)
        eight_fold_list = eight_fold_orbital_order(new_orbital_order)
        print("Eight fold list:", eight_fold_list)
        G_pqrs_new = MajoranaOperator.zero()
        for i, orbital_order in enumerate(eight_fold_list):
            X_pq = make_excitation_operator(orbital_order[0], orbital_order[1])
            X_rs = make_excitation_operator(orbital_order[2], orbital_order[3])
            G_pqrs_new += X_pq*X_rs
        print(G_pqrs_new)


        # Check that all commutators are zero
        all_commutators_zero = True
        num_non_zero_commutators = 0
        for i, item_1 in enumerate(G_pqrs_new.terms.items()):
            term_1, coeff_1 = item_1
            op_1 = MajoranaOperator(term=term_1, coefficient=coeff_1)
            for j, item_2 in enumerate(G_pqrs_new.terms.items()):
                term_2, coeff_2 = item_2
                op_2 = MajoranaOperator(term=term_2, coefficient=coeff_2)
                commutator = op_1*op_2 - op_2*op_1
                if commutator != MajoranaOperator.zero():
                    all_commutators_zero = False
                    num_non_zero_commutators += 1
                    global_all_commutators_zero = False
                    global_num_non_zero_commutators += 1
                # print("Commutator with term", j, ":", term_2, ":", coeff_2, ":", commutator)
        print(f"num_non_zero_commutators = {num_non_zero_commutators}")
        print(all_commutators_zero)
        print("-----")
        
print(f"global_all_commutators_zero = {global_all_commutators_zero}")
print(f"global_num_non_zero_commutators = {global_num_non_zero_commutators}")



Combination: ('p',)
New orbital order: ['p' 'q' 'r' 's']
Eight fold list: [('p', 'q', 'r', 's'), ('p', 'q', 's', 'r'), ('q', 'p', 'r', 's'), ('q', 'p', 's', 'r'), ('r', 's', 'p', 'q'), ('r', 's', 'q', 'p'), ('s', 'r', 'p', 'q'), ('s', 'r', 'q', 'p')]
(-0.5+0j) (0, 3, 4, 7) +
(0.5+0j) (0, 3, 5, 6) +
(0.5+0j) (1, 2, 4, 7) +
(-0.5+0j) (1, 2, 5, 6)
num_non_zero_commutators = 0
True
-----
Combination: ('q',)
New orbital order: ['p' 'q' 'r' 's']
Eight fold list: [('p', 'q', 'r', 's'), ('p', 'q', 's', 'r'), ('q', 'p', 'r', 's'), ('q', 'p', 's', 'r'), ('r', 's', 'p', 'q'), ('r', 's', 'q', 'p'), ('s', 'r', 'p', 'q'), ('s', 'r', 'q', 'p')]
(-0.5+0j) (0, 3, 4, 7) +
(0.5+0j) (0, 3, 5, 6) +
(0.5+0j) (1, 2, 4, 7) +
(-0.5+0j) (1, 2, 5, 6)
num_non_zero_commutators = 0
True
-----
Combination: ('r',)
New orbital order: ['p' 'q' 'r' 's']
Eight fold list: [('p', 'q', 'r', 's'), ('p', 'q', 's', 'r'), ('q', 'p', 'r', 's'), ('q', 'p', 's', 'r'), ('r', 's', 'p', 'q'), ('r', 's', 'q', 'p'), ('s', 'r', 'p', 'q'

# Mutual commutativity between G^PR_QR and G^TR_UR 

In [5]:
global_all_commutators_zero = True
global_num_non_zero_commutators = 0
commutators_not_zero_list = []

orbital_indices_0 = ["p", "q", "r", "r"]
orbital_indices_1 = ["t", "u", "r", "r"]

eight_fold_list_1 = eight_fold_orbital_order(orbital_indices_0)
eight_fold_list_2 = eight_fold_orbital_order(orbital_indices_1)
G_pqrs_new_1 = MajoranaOperator.zero()
G_pqrs_new_2 = MajoranaOperator.zero()
for i, orbital_order in enumerate(eight_fold_list_1):
    X_pq = make_excitation_operator(orbital_order[0], orbital_order[1])
    X_rs = make_excitation_operator(orbital_order[2], orbital_order[3])
    G_pqrs_new_1 += X_pq * X_rs
for i, orbital_order in enumerate(eight_fold_list_2):
    X_pq = make_excitation_operator(orbital_order[0], orbital_order[1])
    X_rs = make_excitation_operator(orbital_order[2], orbital_order[3])
    G_pqrs_new_2 += X_pq * X_rs
print(G_pqrs_new_1)
print(G_pqrs_new_2)

# Check that all commutators are zero
all_commutators_zero = True
num_non_zero_commutators = 0
for i, item_1 in enumerate(G_pqrs_new_1.terms.items()):
    term_1, coeff_1 = item_1
    op_1 = MajoranaOperator(term=term_1, coefficient=coeff_1)
    for j, item_2 in enumerate(G_pqrs_new_2.terms.items()):
        term_2, coeff_2 = item_2
        op_2 = MajoranaOperator(term=term_2, coefficient=coeff_2)
        commutator = op_1 * op_2 - op_2 * op_1
        if commutator != MajoranaOperator.zero():
            all_commutators_zero = False
            num_non_zero_commutators += 1
            global_all_commutators_zero = False
            global_num_non_zero_commutators += 1
        # print("Commutator with term", j, ":", term_2, ":", coeff_2, ":", commutator)
print(f"num_non_zero_commutators = {num_non_zero_commutators}")
print(all_commutators_zero)
print("-----")

print(f"global_all_commutators_zero = {global_all_commutators_zero}")
print(f"global_num_non_zero_commutators = {global_num_non_zero_commutators}")

1j (0, 3) +
(-1+0j) (0, 3, 4, 5) +
-1j (1, 2) +
(1+0j) (1, 2, 4, 5)
(-1+0j) (4, 5, 8, 11) +
(1+0j) (4, 5, 9, 10) +
1j (8, 11) +
-1j (9, 10)
num_non_zero_commutators = 0
True
-----
global_all_commutators_zero = True
global_num_non_zero_commutators = 0


# Mutual commutativity between G^PQ_QS and G^QU_QT 

In [6]:
global_all_commutators_zero = True
global_num_non_zero_commutators = 0
commutators_not_zero_list = []

orbital_indices_0 = ["p", "q", "q", "s"]
orbital_indices_1 = ["q", "q", "u", "t"]

eight_fold_list_1 = eight_fold_orbital_order(orbital_indices_0)
eight_fold_list_2 = eight_fold_orbital_order(orbital_indices_1)
G_pqrs_new_1 = MajoranaOperator.zero()
G_pqrs_new_2 = MajoranaOperator.zero()
for i, orbital_order in enumerate(eight_fold_list_1):
    X_pq = make_excitation_operator(orbital_order[0], orbital_order[1])
    X_rs = make_excitation_operator(orbital_order[2], orbital_order[3])
    G_pqrs_new_1 += X_pq * X_rs
for i, orbital_order in enumerate(eight_fold_list_2):
    X_pq = make_excitation_operator(orbital_order[0], orbital_order[1])
    X_rs = make_excitation_operator(orbital_order[2], orbital_order[3])
    G_pqrs_new_2 += X_pq * X_rs
print(G_pqrs_new_1)
print(G_pqrs_new_2)

# Check that all commutators are zero
all_commutators_zero = True
num_non_zero_commutators = 0
for i, item_1 in enumerate(G_pqrs_new_1.terms.items()):
    term_1, coeff_1 = item_1
    op_1 = MajoranaOperator(term=term_1, coefficient=coeff_1)
    for j, item_2 in enumerate(G_pqrs_new_2.terms.items()):
        term_2, coeff_2 = item_2
        op_2 = MajoranaOperator(term=term_2, coefficient=coeff_2)
        commutator = op_1 * op_2 - op_2 * op_1
        if commutator != MajoranaOperator.zero():
            all_commutators_zero = False
            num_non_zero_commutators += 1
            global_all_commutators_zero = False
            global_num_non_zero_commutators += 1
        # print("Commutator with term", j, ":", term_2, ":", coeff_2, ":", commutator)
print(f"num_non_zero_commutators = {num_non_zero_commutators}")
print(all_commutators_zero)
print("-----")

print(f"global_all_commutators_zero = {global_all_commutators_zero}")
print(f"global_num_non_zero_commutators = {global_num_non_zero_commutators}")

(0.5+0j) (0, 2, 3, 7) +
(-0.5+0j) (1, 2, 3, 6)
(-1+0j) (2, 3, 8, 11) +
(1+0j) (2, 3, 9, 10) +
1j (8, 11) +
-1j (9, 10)
num_non_zero_commutators = 0
True
-----
global_all_commutators_zero = True
global_num_non_zero_commutators = 0
